In [ ]:
# !wget "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X3.zip"
# !unzip DIV2K_train_LR_bicubic_X3.zip

# !wget "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X3.zip"
# !unzip DIV2K_valid_LR_bicubic_X3.zip 

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import glob
# from tqdm import tqdm


# imgs = glob.glob("./DIV2K_train_LR_bicubic/X3/*.png")
# for i in tqdm(range(len(imgs))):
#   img = plt.imread(imgs[i])
#   plt.imsave(f'./drive/My Drive/Image SR/dataset/train/X3/{i}.png', img)

# imgs = glob.glob("./DIV2K_valid_LR_bicubic/X3/*.png")
# for i in tqdm(range(len(imgs))):
#   img = plt.imread(imgs[i])
#   plt.imsave(f'./drive/My Drive/Image SR/dataset/test/X3/{i}.png', img)

In [27]:
import cv2
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle, randint
import numpy as np
import tensorflow as tf
from tensorflow import keras

 
 
vgg19 = keras.applications.VGG19(input_shape=(512,512,3), include_top=False, weights="imagenet")
loss_model = keras.models.Model(vgg19.input, vgg19.get_layer("block5_conv4").output)
tf.test.gpu_device_name()

'/device:GPU:0'

In [28]:
def conv2d(layer_input, filters, name, f_size=4, bn=True):
        """Layers used during downsampling"""
        d = keras.layers.Conv2D(filters, kernel_size=f_size, strides=2, padding='same', name = name+'_c')(layer_input)
        d = keras.layers.LeakyReLU(alpha=0.2, name = name+'_a')(d)
        if bn:
            d = keras.layers.BatchNormalization(momentum=0.8, name = name+'_bn')(d)
        return d
 
def deconv2d(layer_input,  skip_input, filters, name, f_size=4, dropout_rate=0):
    """Layers used during upsampling"""
    u = keras.layers.UpSampling2D(size=2, name = name+'_up')(layer_input)
    u = keras.layers.Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu',name = name+'_c')(u)
    if dropout_rate:
        u = keras.layers.Dropout(dropout_rate, name = name+'_dout')(u)
    #u = keras.layers.BatchNormalization(momentum=0.8, name = name+'_bn')(u)
    u = keras.layers.Concatenate( name = name+'_con')([u, skip_input])
    return u

In [29]:
def get_downscale_size(img, scale=2):
    height, width = img.shape[0], img.shape[1]
    width, height = int(width//scale), int(height//scale)
    if width % 2 == 1:
        width += 1
    if height % 2 == 1:
        height += 1
    return width, height
 
def get_upscale_size(img, scale=2):
    height, width = img.shape[0], img.shape[1]
    return int(width*scale), int(height*scale) 
 
def get_random_crop(image, crop_height, crop_width):
    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height
    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)
    crop = image[y: y + crop_height, x: x + crop_width]
    return crop

In [30]:
def create_generator(input_shape = ( None, None, 3), summary=True, name="generator"):
    
    inputs = keras.layers.Input(shape=input_shape, name="inputs")
    
    d_sample1 = conv2d(inputs, 32, "d_sample1", (3,3), bn=False)
    d_sample2 = conv2d(d_sample1, 64, "d_sample2", (3,3), bn=False)
    d_sample3 = conv2d(d_sample2, 128, "d_sample3", (3,3), bn=False)
    d_sample4 = conv2d(d_sample3, 128, "d_sample4", (3,3), bn=False)

    
    u_sample = deconv2d(d_sample4, d_sample3, 64, "u_sample1", (3,3))
    u_sample = deconv2d(u_sample, d_sample2, 64, "u_sample2", (3,3))
    u_sample = deconv2d(u_sample, d_sample1, 32, "u_sample3", (3,3))


    outputs = deconv2d(u_sample, inputs, 32, "u_sample5", (3,3))
    outputs = keras.layers.Conv2D(3, kernel_size=3, padding="same", name="outputs")(outputs)
    
    model = keras.models.Model(inputs, outputs, name=name)
    None if not summary else model.summary()
    return model
 

@tf.function
def loss_fn(target, predicted):    
    vgg_loss = keras.metrics.MSE(loss_model(target), loss_model(predicted))
    mse_loss = keras.metrics.MSE(target, predicted)
    loss = 10*keras.backend.mean(vgg_loss) + keras.backend.mean(mse_loss)   
    return loss

In [31]:
def create_discriminator(input_shape=(512, 512, 3), summary=True):
    inputs = keras.layers.Input(shape=input_shape)
    
    conv1 = conv2d(inputs, 16, "conv1", (3,3), bn=False)
    max_pool = keras.layers.MaxPool2D(2, name="max_pool1")(conv1)
    conv2 = conv2d(max_pool, 32, "conv2", (3,3), bn=False)
    max_pool = keras.layers.MaxPool2D(2, name="max_pool2")(conv2)
    conv3 = conv2d(max_pool, 64, "conv3", (3,3), bn=False)
    max_pool = keras.layers.MaxPool2D(2, name="max_pool3")(conv3)
    conv4 = conv2d(max_pool, 128, "conv4", (3,3), bn=False)
    max_pool = keras.layers.MaxPool2D(2, name="max_pool4")(conv4)
    flatten = keras.layers.Flatten(name="flatten")(max_pool)
    dense1 = keras.layers.Dense(128, activation="relu", name="dense1")(flatten)
    outputs = keras.layers.Dense(1, activation="sigmoid", name="outputs")(dense1)
    
    model = keras.models.Model(inputs, outputs)
    None if not summary else model.summary()
    return model
 
 
def train_discriminator(discriminator, real, fake):
    discriminator.trainable = True
    ones, zeros = np.ones(len(real)), np.zeros(len(fake))
    outputs = np.concatenate((ones, zeros))
    inputs = np.concatenate((real, fake))
    random_shuffle_index = np.random.permutation(len(outputs))
    outputs, inputs = outputs[random_shuffle_index], inputs[random_shuffle_index]
    loss, accuracy = discriminator.train_on_batch(inputs, outputs)
    return loss, accuracy

In [32]:
def g_on_d(discriminator, generator):
    inputs = keras.layers.Input(shape=(512,512,3))
    fake = generator(inputs)
    outputs = discriminator(fake)
    
    model = keras.models.Model(inputs=inputs, outputs=[fake, outputs])
    model.summary()
    
    return model

In [33]:
def train_gan(gan, generator, discriminator, 
              imgs, epochs=1, batch_size=10, 
              train_disc = True,
              generator_name="model_generator", 
              discriminator_name="model_discriminator"):
    n = len(imgs)    
    for epoch in range(epochs):
        shuffle(imgs)
        t = tqdm(range(0, n, batch_size))
        t.desc = f"Epoch {epoch+1}/{epochs} -"
        d_loss, accuracy = 0, 0
        for i in t:            
            real = [get_random_crop(img, 512, 512) for img in imgs[i:i+batch_size]]
            inputs = keras.backend.variable([cv2.resize(cv2.resize(img, (128,128)), (512,512), cv2.INTER_CUBIC) for img in real])
            if train_disc:
                fake = generator.predict_on_batch(inputs)
                discriminator.trainable = True
                d_loss, accuracy = train_discriminator(discriminator, real, fake)
            discriminator.trainable = False
 
            g_loss = gan.train_on_batch(inputs, [tf.keras.backend.variable(real), np.ones(len(real)) ])
            t.postfix = f"d_loss: {d_loss:.5f} a: {accuracy*100:.2f}% g_loss: [{g_loss[0]:.5f} {g_loss[1]:.5f} {g_loss[2]:.5f}] "
            
        discriminator.save(f"./drive/My Drive/Image SR/2/{discriminator_name}_4x_gan.h5")
        generator.save(f"./drive/My Drive/Image SR/2/{generator_name}_4x_gan.h5")

In [34]:
def predict(model, imgs):
    targets = [cv2.resize(img, (512, 512), cv2.INTER_AREA) for img in imgs]
    frames = [cv2.resize(img, (128, 128)) for img in targets]
    frames = [cv2.resize(img, (512, 512), cv2.INTER_CUBIC) for img in frames]
    
    predicted = model.predict_on_batch(keras.backend.variable(frames))
    return targets, frames, predicted

In [35]:
discriminator = create_discriminator(summary=False)
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
discriminator.load_weights('./drive/My Drive/Image SR/2/model_discriminator_4x_gan.h5')


# generator = create_generator(summary = False)
generator = create_generator((512,512,3), False)
generator.compile(optimizer="adam", loss=loss_fn)
generator.load_weights('./drive/My Drive/Image SR/2/model_generator_4x_gan.h5')
 
discriminator.trainable = False
gan = g_on_d(discriminator, generator)
gan.compile(optimizer="adam", loss=[loss_fn, "binary_crossentropy"], 
            loss_weights=[100,7])
discriminator.trainable = True

Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 512, 512, 3)]     0         
_________________________________________________________________
generator (Functional)       (None, 512, 512, 3)       481588    
_________________________________________________________________
functional_19 (Functional)   (None, 1)                 163233    
Total params: 644,821
Trainable params: 481,588
Non-trainable params: 163,233
_________________________________________________________________


In [36]:
train_imgs = glob.glob('./DIV2K_train_LR_bicubic/X3/*.png')
train_imgs = [cv2.resize(plt.imread(img)[:,:,:3], (600,600), cv2.INTER_AREA) for img in tqdm(train_imgs)]
test_imgs = glob.glob('./drive/My Drive/Image SR/dataset/test/X3/*.png')[:20]
test_imgs = [cv2.resize(plt.imread(img)[:,:,:3], (600,600), cv2.INTER_AREA) for img in tqdm(test_imgs)]

100%|██████████| 20/20 [00:00<00:00, 24.90it/s]


In [37]:
epoch_no = 78

In [39]:
for i in range(2):
    train_gan(gan, generator, discriminator, train_imgs, epochs=1, 
              train_disc=True, batch_size=10,
              generator_name="model_generator", 
              discriminator_name="model_discriminator"
        )
       
    targets, frames, predicted = predict(generator, test_imgs)  
    
    for j in range(len(frames)):
      img = np.zeros(shape=(512, 512*3, 3))
      img[:, :512, :] = targets[j]
      img[:,512:1024, :] = cv2.resize(frames[j], (512,512), cv2.INTER_CUBIC)
      img[:,1024:, :] = predicted[j]
      img[:,1024:, :][img[:,1024:, :]>1] = 1
      img[:,1024:, :][img[:,1024:, :]<0] = 0
      plt.imsave(f"./drive/My Drive/Image SR/2/results_1/{j+1}_{epoch_no}.png",img)
    epoch_no +=1

Epoch 1/1 -: 100%|██████████| 80/80 [12:15<00:00,  9.19s/it, d_loss: 0.69529 a: 50.00% g_loss: [17.53845 0.13024 0.64487] ]


In [ ]:
train_gan(gan, generator, discriminator, train_imgs, epochs=1, 
              train_disc=True, batch_size=3,
              generator_name="model_generator", 
              discriminator_name="model_discriminator"
        )

In [ ]:
targets, frames, predicted = predict(generator, test_imgs)  
    
for j in range(len(frames)):
  img = np.zeros(shape=(512, 512*3, 3))
  img[:, :512, :] = targets[j]
  img[:,512:1024, :] = cv2.resize(frames[j], (512,512), cv2.INTER_CUBIC)
  img[:,1024:, :] = predicted[j]
  img[:,1024:, :][img[:,1024:, :]>1] = 1
  img[:,1024:, :][img[:,1024:, :]<0] = 0
  plt.imsave(f"./drive/My Drive/Image SR/2/results_1/{j+1}_{epoch_no}.png",img)

In [ ]:
test_imgs = glob.glob('./drive/My Drive/Image SR/dataset/test/X3/*.png')[:20]
test_imgs = [plt.imread(img)[:,:,:3] for img in tqdm(test_imgs)]

100%|██████████| 20/20 [00:00<00:00, 33.50it/s]


In [ ]:
pos = 20
width, height = test_imgs[pos-1].shape[0], test_imgs[pos-1].shape[1]
img = cv2.resize(test_imgs[pos-1], (int((height+8-height%8)*2), int((width+8-width%8)*2)))
# img = cv2.resize(test_imgs[pos-1], (int(height-height%16), int(width-width%16)))
predicted_img = generator.predict_on_batch(keras.backend.variable([  img  ]))

In [ ]:
temp = predicted_img[0]
temp[temp > 1] = 1
temp[temp < 0] = 0
# 
plt.imsave(f"{pos}_1.png",temp)

In [ ]:
img = cv2.resize(predicted_img[0], (int((height+8-height%8)*2), int((width+8-width%8)*2)))
predicted_img = generator.predict_on_batch(keras.backend.variable([  img  ]))

In [ ]:
temp = predicted_img[0]
temp[temp > 1] = 1
temp[temp < 0] = 0
# 
plt.imsave(f"{pos}_2.png",temp)

In [ ]:
img.shape

(912, 1376, 3)

In [ ]:
predicted_img.shape

(1, 912, 1376, 3)

In [3]:
import time

In [6]:
time.ctime()

'Wed Aug 12 23:55:04 2020'

In [9]:
import datetime

datetime.time(0, 0)